In [ ]:
# take a few frames while tracking targets

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS

In [ ]:
# for tab completion to work
%config IPCompleter.use_jedi = False

In [ ]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
#get classes and start them
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
tmp = atcs.rem.atptg.evt_summaryState.get()
print(salobj.State(tmp.summaryState))

In [ ]:
tmp = await atcs.rem.atdome.evt_heartbeat.next(flush=True)
print(tmp)

In [ ]:
tmp =latiss.rem.atspectrograph.evt_settingsApplied.get()
print(tmp)

In [ ]:
#await atcs.enable()
await latiss.enable()

In [ ]:
tmp = latiss.rem.atarchiver.evt_heartbeat.get()
print(tmp)

In [ ]:
tmp = latiss.rem.atarchiver.evt_summaryState.get()
print(salobj.State(tmp.summaryState))

In [ ]:
# enable components
#await atcs.enable()
#await latiss.enable()

In [ ]:
# ATDOME only
tmp = await salobj.set_summary_state(atcs.rem.atdome, salobj.State.STANDBY, settingsToApply='test')
tmp = await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.STANDBY)
#tmp=await atcs.rem.atdome.cmd_start.set_start(settingsToApply='test')
#tmp=await atcs.rem.atdome.cmd_enable.start()
#print(tmp)

In [ ]:
# ATMCS
tmp = await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)
#tmp=await atcs.rem.atmcs.cmd_start.start(timeout=5)

In [ ]:
# ATHexapod
tmp = await salobj.set_summary_state(atcs.rem.athexapod, salobj.State.ENABLED, settingsToApply='summit')
#tmp = await atcs.rem.athexapod.cmd_start.set_start(settingsToApply='auxtel_201912.yaml')

In [ ]:
# ATPneumatics
tmp = await salobj.set_summary_state(atcs.rem.atpneumatics, salobj.State.STANDBY)
#tmp=await atcs.rem.atpneumatics.cmd_start.start()

In [ ]:
# ATPtg
tmp = await salobj.set_summary_state(atcs.rem.atptg, salobj.State.STANDBY)
#tmp=await atcs.rem.atptg.cmd_enable.start()
# Need to manually say which rotator we're using at the moment (N2-->focus=3, N1-->focus=2)
#await atcs.rem.atptg.cmd_focusName.set_start(focus=3)

In [ ]:
# ATAOS
tmp = await salobj.set_summary_state(atcs.rem.ataos, salobj.State.STANDBY)
#tmp=await atcs.rem.ataos.cmd_enable.start()

In [ ]:
await atcs.rem.atdome.cmd_homeAzimuth.start()

In [ ]:
# This doesn't move anything! 
tmp=await atcs.rem.atdome.cmd_moveAzimuth.set_start(5)
print(tmp)

In [ ]:
# This does
tmp=await atcs.rem.atdome.cmd_moveAzimuth.set_start(azimuth=0)
print(tmp)

In [ ]:
# ATHexapod
tmp = await salobj.set_summary_state(atcs.rem.athexapod, salobj.State.STANDBY, settingsToApply='summit')
#tmp=await atcs.rem.athexapod.cmd_start.set_start(settingsToApply='summit')
#tmp=await atcs.rem.athexapod.cmd_enable.start()
print(tmp)

In [ ]:
# ATSPECTROGRAPH
tmp = await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.ENABLED, settingsToApply='LATISS_AuxTel_20210108_v3.yaml')
#tmp=await latiss.rem.atspectrograph.cmd_start.set_start(settingsToApply='LATISS_AuxTel_20210108.yaml')
#tmp=await latiss.rem.atspectrograph.cmd_enable.start()
print(tmp)

In [ ]:
# take event checking out the slew commands to test telescope only
atcs.check.atdome = False
atcs.check.atdometrajectory = False

In [ ]:
from datetime import datetime

current_time = datetime.now().strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")

In [ ]:
# turn on corrections
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
await atcs.slew_object('HR 8500')

In [ ]:
await latiss.take_object(exptime=2, grating='empty_1', filter='RG610')

In [ ]:
await latiss.take_object(exptime=1, grating='ronchi90lpmm', filter='RG610')

In [ ]:
await atcs.stop_tracking()

In [ ]:
await atcs.standby()
await latiss.standby()

In [ ]:
await atcs.rem.atpneumatics.cmd_start.start()

In [ ]:
await atcs.rem.atpneumatics.cmd_enable.start()

In [ ]:
await atcs.rem.ataos.cmd_start.start()

In [ ]:
await atcs.rem.ataos.cmd_enable.start()

In [ ]:
# turn off corrections
tmp = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
while True:
    await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')
    await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')

In [ ]:
await atcs.slew_object('HR 280')

In [ ]:
# turn on corrections
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# turn off corrections
tmp = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# is ATAOS correcting?
tmp = atcs.rem.ataos.evt_correctionEnabled.get()
print(tmp)

In [ ]:
# get offsets from ATAOS - after turning on atspectrograph correction
# this had a filter in the beam? why is it not showing?  -- I think the display was just stale
focus_offsets = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(focus_offsets)

In [ ]:
# Verify that ATAOS handles focus/pointing offsets from grating change
await latiss.rem.atspectrograph.cmd_changeFilter.set_start(filter=0)
await latiss.rem.atspectrograph.cmd_changeDisperser.set_start(disperser=0)

In [ ]:
# get offsets from ATAOS - after turning on atspectrograph correction
# this had a filter in the beam? why is it not showing?
focus_offsets = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(focus_offsets)

In [ ]:
# Verify that ATAOS handles focus/pointing offsets from grating change
await latiss.rem.atspectrograph.cmd_changeFilter.set_start(filter=0)
await latiss.rem.atspectrograph.cmd_changeDisperser.set_start(disperser=1)

In [ ]:
# get offsets from ATAOS - after turning on atspectrograph correction
# this had a filter in the beam? why is it not showing?
focus_offsets = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(focus_offsets)

In [ ]:
# get pointing offsets from ATAOS
pointing_offsets = await atcs.rem.ataos.evt_pointingOffsetSummary.aget()
print(pointing_offsets)

In [ ]:
# now apply user offset
await atcs.rem.ataos.cmd_offset.set_start(z=1.34)

In [ ]:
# get pointing offsets from ATAOS
pointing_offsets = atcs.rem.ataos.evt_pointingOffsetSummary.get()
print(pointing_offsets)

In [ ]:
# reset offsets
tmp = await atcs.rem.ataos.cmd_resetOffset.set_start(axis='z')

In [ ]:
# take LATISS images
await latiss.take_bias(nbias=3)

In [ ]:
# get offsets from ATAOS
focus_offsets = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(focus_offsets)

In [ ]:
await latiss.setup_atspec(filter=0, grating=0)

In [ ]:
await latiss.take_engtest(exptime=1)